https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization

In [1]:
import functools
import os
CWD = os.getcwd()
from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import cv2
import tensorflow as tf
import keras
import tensorflow_hub as hub
import random
import derm_ita as dita
print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))
print(CWD)

2024-02-12 17:45:07.401335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-12 17:45:07.401386: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-12 17:45:07.402078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-12 17:45:07.407508: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TF Version:  2.15.0
TF Hub version:  0.15.0
Eager mode enabled:  True
GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/mnt/e/ap_research/datasets/ISIC_ArGI


2024-02-12 17:45:08.722263: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-12 17:45:08.748286: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-12 17:45:08.748335: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:0a:00.0/numa_node
Your kernel may have been built without NUMA support.


In [25]:
# Load TF Hub module.

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

In [3]:
def crop_center(image):
  """Returns a cropped square image."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

@functools.lru_cache(maxsize=None)
def load_image(PATH, image_size=(256, 256), preserve_aspect_ratio=True):
  """Loads and preprocesses images."""
  # Cache image file locally.
  # Load and convert to float32 numpy array, add batch dimension, and normalize to range [0, 1].
  img = tf.io.decode_image(
      tf.io.read_file(PATH),
      channels=3, dtype=tf.float32)[tf.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [4]:
def stylize(content_image, style_image):
    outputs = hub_module(content_image, style_image)
    stylized_image = outputs[0]
    # Stylize content image with given style image.
    # This is pretty fast within a few milliseconds on a GPU.
    outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
    # print(outputs)
    stylized_image = outputs[0]
    # Visualize input images and the generated stylized image.
    # show_n([content_image, style_image, stylized_image], titles=['Original content image', 'Style image', 'Stylized image'])
    stylized_image = tf.squeeze(stylized_image)
    return stylized_image

Randomly Select 656 ISIC_2017 benign images for content

In [19]:
def cd(path):
    index = 0
    for i in range(len(path)):
        if path[len(path)-i-1] == '/':
            index = len(path)-i-1
            break
    return path[0:index]

In [21]:
path_content = f'{cd(CWD)}/ISIC/all_images_original'

In [37]:
content_images = os.listdir(path_content)

In [26]:
def choose(min, max, count):
    indexes = set()
    while len(indexes) < count:
        i = random.randint(min,max)
        indexes.add(i)    
    return list(indexes)

In [27]:
metadata_ISIC_ArGI = pd.read_excel(f'{CWD}/metadata_ISIC_ArGI.xlsx', index_col='index')
num_imgs = len(metadata_ISIC_ArGI['id'])
print(metadata_ISIC_ArGI)


                    id      label  malignance  skin_tone
index                                                   
0      ISIC_000000.jpg     benign           0         12
1      ISIC_000001.jpg     benign           0         12
2      ISIC_000002.jpg  malignant           1         12
3      ISIC_000003.jpg     benign           0         12
4      ISIC_000004.jpg  malignant           1         56
...                ...        ...         ...        ...
1971   ISIC_015251.jpg  malignant           1         12
1972   ISIC_015256.jpg  malignant           1         34
1973   ISIC_015260.jpg     benign           0         12
1974   ISIC_015284.jpg  malignant           1         12
1975   ISIC_015295.jpg     benign           0         56

[1976 rows x 4 columns]


In [28]:
images = np.empty(shape=[num_imgs+656, 64, 64, 3], dtype=np.uint8)
for i in range(num_imgs):
    id = metadata_ISIC_ArGI['id'][i]
    image = tf.keras.utils.load_img(f'{CWD}/all_images/{id}', target_size = (64, 64))
    images[i] = image

In [29]:
print(len(images))

2632


In [31]:
def ITA_FST(image):
    img_ita = dita.get_cropped_center_ita(image)
    groh_FST = None
    # Following the standard set in DDI.
    if 23 <= img_ita:
        groh_FST = 12
    elif 0 <= img_ita < 23:
        groh_FST = 34
    elif img_ita < 0:
        groh_FST = 56

    return groh_FST

In [40]:
indexes = choose(0, 1976, 656)
print(indexes)

[5, 7, 8, 9, 10, 11, 13, 15, 20, 24, 25, 26, 33, 35, 37, 43, 46, 50, 57, 60, 61, 64, 66, 68, 70, 72, 78, 79, 82, 83, 84, 87, 90, 95, 99, 101, 110, 115, 116, 119, 120, 122, 127, 131, 133, 143, 145, 147, 174, 176, 177, 179, 185, 190, 191, 192, 194, 196, 198, 199, 205, 206, 207, 208, 210, 212, 213, 216, 218, 223, 232, 237, 240, 241, 242, 243, 245, 247, 250, 255, 256, 257, 259, 263, 264, 265, 267, 270, 272, 274, 275, 279, 283, 287, 288, 293, 304, 309, 310, 313, 314, 317, 320, 327, 329, 330, 331, 337, 339, 340, 349, 354, 356, 358, 362, 363, 365, 367, 369, 370, 378, 379, 381, 387, 388, 392, 394, 395, 402, 409, 412, 417, 419, 420, 424, 428, 429, 430, 431, 434, 437, 438, 444, 445, 446, 448, 451, 453, 460, 462, 463, 465, 466, 468, 469, 479, 481, 485, 488, 490, 496, 499, 500, 501, 502, 505, 510, 515, 517, 523, 528, 529, 532, 533, 534, 536, 537, 538, 539, 550, 552, 556, 558, 563, 573, 575, 577, 578, 579, 580, 583, 589, 592, 593, 598, 600, 607, 609, 610, 612, 616, 617, 619, 624, 626, 627, 628, 634

In [44]:
ids = list(metadata_ISIC_ArGI['id'])
labels = list(metadata_ISIC_ArGI['label'])
malignances = list(metadata_ISIC_ArGI['malignance'])
skin_tones = list(metadata_ISIC_ArGI['skin_tone'])
style_images = os.listdir(f'{CWD}/style_images')

for i in range(656):
    index = indexes[i]
    idx = random.randint(0,43)
    style_image = load_image(f"{CWD}/style_images/{style_images[idx]}", (256,256))
    style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
    print(f'{path_content}/{content_images[index]}')
    content_image = load_image(f'{path_content}/{content_images[index]}', (224, 224))

    content_id = metadata_ISIC_ArGI['id'][index]
    style_id = style_images[idx]

    print(f'Content ID: {content_id}')
    print(f'Style ID: {style_id}')

    new_filename = f"{content_id}stylized_with_{style_id}.jpg"
    new_img = stylize(content_image, style_image)
    ids.append(new_filename)
    labels.append(metadata_ISIC_ArGI['label'][index])
    malignances.append(metadata_ISIC_ArGI['malignance'][index])

    destination_path = f"{CWD}/all_images/{new_filename}"

    keras.utils.save_img(destination_path, new_img) 
    new_img = tf.keras.utils.load_img(destination_path, target_size=(64,64))
    skin_tones.append(ITA_FST(new_img))
    images[1976+i] = new_img
    
    

/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000006.jpg
Content ID: ISIC_000006.jpg
Style ID: img12.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000008.jpg
Content ID: ISIC_000008.jpg
Style ID: img30.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000009.jpg
Content ID: ISIC_000009.jpg
Style ID: img3.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000010.jpg
Content ID: ISIC_000010.jpg
Style ID: img32.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000011.jpg
Content ID: ISIC_000011.jpg
Style ID: img44.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000012.jpg
Content ID: ISIC_000012.jpg
Style ID: img6.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000014.jpg
Content ID: ISIC_000014.jpg
Style ID: img33.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_0000016.jpg
Content ID: ISIC_000016.jpg
Style ID: img30.jpg
/mnt/e/ap_research/datasets/ISIC/all_images_original/ISIC_

In [45]:
print(ids)
print(labels)
print(malignances)
print(skin_tones)

['ISIC_000000.jpg', 'ISIC_000001.jpg', 'ISIC_000002.jpg', 'ISIC_000003.jpg', 'ISIC_000004.jpg', 'ISIC_000006.jpg', 'ISIC_000007.jpg', 'ISIC_000008.jpg', 'ISIC_000009.jpg', 'ISIC_000010.jpg', 'ISIC_000011.jpg', 'ISIC_000012.jpg', 'ISIC_000013.jpg', 'ISIC_000014.jpg', 'ISIC_000015.jpg', 'ISIC_000016.jpg', 'ISIC_000017.jpg', 'ISIC_000018.jpg', 'ISIC_000019.jpg', 'ISIC_000020.jpg', 'ISIC_000021.jpg', 'ISIC_000022.jpg', 'ISIC_000023.jpg', 'ISIC_000024.jpg', 'ISIC_000025.jpg', 'ISIC_000026.jpg', 'ISIC_000027.jpg', 'ISIC_000028.jpg', 'ISIC_000029.jpg', 'ISIC_000030.jpg', 'ISIC_000031.jpg', 'ISIC_000032.jpg', 'ISIC_000034.jpg', 'ISIC_000035.jpg', 'ISIC_000036.jpg', 'ISIC_000037.jpg', 'ISIC_000038.jpg', 'ISIC_000039.jpg', 'ISIC_000040.jpg', 'ISIC_000041.jpg', 'ISIC_000042.jpg', 'ISIC_000043.jpg', 'ISIC_000044.jpg', 'ISIC_000045.jpg', 'ISIC_000046.jpg', 'ISIC_000047.jpg', 'ISIC_000048.jpg', 'ISIC_000049.jpg', 'ISIC_000050.jpg', 'ISIC_000051.jpg', 'ISIC_000052.jpg', 'ISIC_000053.jpg', 'ISIC_00005

In [46]:
print(len(ids))
print(len(labels))
print(len(malignances))
print(len(skin_tones))

2632
2632
2632
2632


In [47]:
new_data = {
    'id':ids,
    'label':labels,
    'malignance': malignances,
    'skin_tone': skin_tones
}
new_metadata = pd.DataFrame.from_dict(new_data)
print(new_metadata)

                                              id      label  malignance  \
0                                ISIC_000000.jpg     benign           0   
1                                ISIC_000001.jpg     benign           0   
2                                ISIC_000002.jpg  malignant           1   
3                                ISIC_000003.jpg     benign           0   
4                                ISIC_000004.jpg  malignant           1   
...                                          ...        ...         ...   
2627  ISIC_015185.jpgstylized_with_img22.jpg.jpg  malignant           1   
2628   ISIC_015204.jpgstylized_with_img5.jpg.jpg  malignant           1   
2629   ISIC_015233.jpgstylized_with_img7.jpg.jpg     benign           0   
2630  ISIC_015256.jpgstylized_with_img22.jpg.jpg  malignant           1   
2631  ISIC_015284.jpgstylized_with_img43.jpg.jpg  malignant           1   

      skin_tone  
0            12  
1            12  
2            12  
3            12  
4        

In [49]:
new_metadata.to_excel(f'{CWD}/metadata_ISIC_ArGI.xlsx')

# Go into Excel and sort the range by id (don't sort the index column!)

In [2]:
METADATA = pd.read_excel(f'{CWD}/metadata_ISIC_ArGI.xlsx' , index_col = 'index')
num_imgs = len(METADATA['id'])
print(METADATA)
print(num_imgs)

                                               id      label  malignance  \
index                                                                      
0                                 ISIC_000000.jpg     benign           0   
1                                 ISIC_000001.jpg     benign           0   
2                                 ISIC_000002.jpg  malignant           1   
3                                 ISIC_000003.jpg     benign           0   
4                                 ISIC_000004.jpg  malignant           1   
...                                           ...        ...         ...   
2627   ISIC_015256.jpgstylized_with_img22.jpg.jpg  malignant           1   
2628                              ISIC_015260.jpg     benign           0   
2629                              ISIC_015284.jpg  malignant           1   
2630   ISIC_015284.jpgstylized_with_img43.jpg.jpg  malignant           1   
2631                              ISIC_015295.jpg     benign           0   

       skin

In [3]:
images = np.empty(shape=[num_imgs, 64, 64, 3], dtype=np.uint8)
for i in range(num_imgs):
    id = METADATA['id'][i]
    image = tf.keras.utils.load_img(f'{CWD}/all_images/{id}', target_size = (64, 64))
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    images[i] = image

In [4]:
print(images.shape)
print(np.array(METADATA['malignance'], dtype=np.uint8).reshape(num_imgs,1).shape)

(2632, 64, 64, 3)
(2632, 1)


In [59]:
import h5py
with h5py.File(f'{CWD}/ISIC_ArGI.h5','w') as f:
    f.create_dataset('images', data = images)
    f.create_dataset('labels', data = np.array(METADATA['malignance'], dtype=np.uint8).reshape(num_imgs,1))